# Pseudochannel Explorer

Interactive tool for creating weighted pseudochannel composites from multiplex tissue imaging data.

## Supported Input Formats
- **Folder of TIFFs**: Individual channel images in a folder (channel names from filenames)
- **OME-TIFF**: Single multi-channel file with marker names from a separate file

## Workflow
1. Load channel images (from folder or OME-TIFF)
2. Interactively tune weights using sliders
3. Preview results in real-time (on downsampled images)
4. Save configuration for reuse
5. Apply to full-resolution images or batch process

## 1. Setup

In [ ]:
import sys
from pathlib import Path

# Add src to path if running from notebooks folder
src_path = Path("../src").resolve()
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from pseudochannel import (
    DEFAULT_EXCLUDED_CHANNELS,
    load_channel_folder,
    load_ome_tiff,
    load_marker_names,
    OMETiffChannels,
    compute_pseudochannel,
    create_preview_stack,
    save_config,
    load_config,
    process_dataset,
)
from pseudochannel.widgets import create_interactive_explorer, PseudochannelExplorer
from pseudochannel.batch import (
    batch_process_directory,
    find_channel_folders,
    process_ome_tiff,
    process_ome_tiff_batch,
)
from pseudochannel.config import get_weights_from_config, list_configs

import numpy as np
import matplotlib.pyplot as plt
import tifffile

# Use widget backend for interactive zoom feature
%matplotlib widget

In [ ]:
# Configuration - set your paths here
# Option A: Folder with individual channel TIFFs
CHANNEL_FOLDER = "/path/to/channel_folder"

# Option B: OME-TIFF with marker file
OME_TIFF_PATH = "/path/to/sample.ome.tiff"
MARKER_FILE = "/path/to/markers.txt"  # One marker name per line
# Or use CSV: MARKER_FILE = "../data/panel.csv" with MARKER_COLUMN = "marker_name"
MARKER_COLUMN = None  # Set to column name/index for CSV files

# Channel exclusion configuration
# Set to None to use default exclusions (DAPI, None, APC, FE, Autofluorescence, etc.)
# Set to [] to include ALL channels
# Set to custom list to exclude specific channels
EXCLUDE_CHANNELS = None  # Uses DEFAULT_EXCLUDED_CHANNELS

# Nuclear marker configuration (for overlay feature)
# For Option A (folder): Path to DAPI/nuclear stain file
# For Option B (OME-TIFF): Leave as None to auto-detect DAPI channel
NUCLEAR_MARKER_PATH = "/path/to/nuclear_marker.tiff"  # or None

# Show default exclusion list
print(f"Default excluded channels: {sorted(DEFAULT_EXCLUDED_CHANNELS)}")

# Common paths
CONFIG_DIR = "/path/to/Pseudochannel_configs"
OUTPUT_DIR = "/path/to/outputs"

## 2. Load Channels

Choose **Option A** (folder) or **Option B** (OME-TIFF) below.

### Option A: Load from Channel Folder

In [ ]:
# Load channel images (memory-mapped for efficiency)
channels = load_channel_folder(CHANNEL_FOLDER, exclude_channels=EXCLUDE_CHANNELS)

print(f"Loaded {len(channels)} channels:")
for name, arr in channels.items():
    print(f"  {name}: shape={arr.shape}, dtype={arr.dtype}")

### Option B: Load from OME-TIFF

In [ ]:
# First, examine the marker file
marker_names = load_marker_names(MARKER_FILE, column=MARKER_COLUMN)
print(f"Found {len(marker_names)} markers:")
for i, name in enumerate(marker_names):
    print(f"  {i}: {name}")

In [ ]:
# Load OME-TIFF with marker names
channels = load_ome_tiff(
    OME_TIFF_PATH,
    MARKER_FILE,
    marker_column=MARKER_COLUMN,
    exclude_channels=EXCLUDE_CHANNELS
)

print(f"Loaded {len(channels)} channels:")
for name, arr in channels.items():
    print(f"  {name}: shape={arr.shape}, dtype={arr.dtype}")

# # Alternative: Use OMETiffChannels for memory-efficient access
# # This keeps the file memory-mapped and provides views into channels
# channels = OMETiffChannels(
#     OME_TIFF_PATH,
#     MARKER_FILE,
#     marker_column=MARKER_COLUMN,
#     exclude_channels=EXCLUDE_CHANNELS
# )
# print(f"Loaded {len(channels)} channels with shape {channels.shape}")

In [ ]:
# Create downsampled preview stack (done once, reused for all slider updates)
previews = create_preview_stack(channels, target_size=512)

print(f"Preview stack created with shape: {next(iter(previews.values())).shape}")

## 3. Interactive Weight Tuning

Use the sliders to adjust the weight of each channel. The preview updates in real-time.

In [ ]:
# OPTION A: Create interactive explorer from folder
# The explorer shows a merged pseudochannel preview that updates with slider values
# Enable "Show Nuclear (DAPI)" checkbox to overlay nuclear marker in blue
explorer = create_interactive_explorer(
    CHANNEL_FOLDER,
    preview_size=512,
    exclude_channels=EXCLUDE_CHANNELS,
    nuclear_marker_path=NUCLEAR_MARKER_PATH
)

# # OPTION B: Alternative: Create explorer directly from OME-TIFF
# # For OME-TIFF, DAPI channel is auto-detected if nuclear_marker_path is not specified
# explorer = create_interactive_explorer(
#     OME_TIFF_PATH, 
#     marker_file=MARKER_FILE,
#     marker_column=MARKER_COLUMN,
#     preview_size=512,
#     exclude_channels=EXCLUDE_CHANNELS
# )

In [ ]:
# Get current weights from sliders
current_weights = explorer.get_weights()
print("Current weights:")
for name, weight in current_weights.items():
    if weight > 0:
        print(f"  {name}: {weight:.2f}")

## 4. Apply to Full Resolution

Once satisfied with weights, compute the full-resolution pseudochannel.

In [ ]:
# Compute full-resolution pseudochannel
weights = explorer.get_weights()

print("Computing full-resolution pseudochannel...")
full_res_result = compute_pseudochannel(
    channels,
    weights,
    normalize="minmax"
)
print(f"Result shape: {full_res_result.shape}")

In [ ]:
# Display full-resolution result (downsampled for display)
from pseudochannel.preview import downsample_image

display_preview = downsample_image(full_res_result, target_size=1024)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(display_preview, cmap='gray')
ax.set_title('Full Resolution Pseudochannel (downsampled for display)')
ax.axis('off')
plt.show()

In [ ]:
# Save full-resolution result as TIFF
output_path = Path(OUTPUT_DIR) / "pseudochannel_output.tif"
output_path.parent.mkdir(parents=True, exist_ok=True)

# Convert to 16-bit for saving
output_16bit = (full_res_result * 65535).astype(np.uint16)
tifffile.imwrite(str(output_path), output_16bit)

print(f"Saved to: {output_path}")

## 5. Save Configuration

Save the weight configuration for reuse with other images.

In [ ]:
# Save configuration
config_name = "my_pseudochannel_config"  # Change this name as needed
config_description = "Custom pseudochannel weights for membrane visualization"

config_path = save_config(
    weights=explorer.get_weights(),
    output_path=Path(CONFIG_DIR) / f"{config_name}.yaml",
    name=config_name,
    description=config_description,
    normalization="minmax"
)

print(f"Configuration saved to: {config_path}")

In [ ]:
# List available configurations
configs = list_configs(CONFIG_DIR)

print("Available configurations:")
for cfg in configs:
    print(f"  {cfg['name']}: {cfg['num_channels']} channels - {cfg['description'][:50]}")

In [ ]:
# Load a saved configuration
config = load_config(Path(CONFIG_DIR) / "my_pseudochannel_config.yaml")
explorer.set_weights(get_weights_from_config(config))

## 6. Batch Processing

Apply the saved configuration to multiple image folders.

In [ ]:
# Find all channel folders in a directory
data_root = Path("../data")
folders = find_channel_folders(data_root)
print(f"Found {len(folders)} channel folders:")
for f in folders:
    print(f"  {f}")

In [ ]:
# Batch process all folders using a saved config
output_paths = batch_process_directory(
    root_path="../data",
    config_path=Path(CONFIG_DIR) / "my_pseudochannel_config.yaml",
    output_folder="../outputs/batch",
)

print(f"Processed {len(output_paths)} folders")

In [ ]:
# Or process specific folders
input_folders = [
    "../data/sample1",
    "../data/sample2",
    "../data/sample3",
]

output_paths = process_dataset(
    input_folders=input_folders,
    config_path=Path(CONFIG_DIR) / "my_pseudochannel_config.yaml",
    output_folder="../outputs/batch",
)

### Batch Process OME-TIFF Files

In [ ]:
# Process multiple OME-TIFF files with a shared marker file
tiff_files = [
    "../data/sample1.ome.tiff",
    "../data/sample2.ome.tiff",
    "../data/sample3.ome.tiff",
]

output_paths = process_ome_tiff_batch(
    tiff_files=tiff_files,
    marker_files="../data/markers.txt",  # Same marker file for all
    config_path=Path(CONFIG_DIR) / "my_pseudochannel_config.yaml",
    output_folder="../outputs/batch",
)

print(f"Processed {len(output_paths)} OME-TIFF files")

## Marker File Formats

The marker file can be in several formats:

### Plain Text (one marker per line)
```
DAPI
CD45
E-cadherin
Pan-CK
```

### CSV with header
```csv
channel_id,marker_name,target
1,DAPI,Nucleus
2,CD45,Immune
3,E-cadherin,Epithelial
```
Use: `marker_column="marker_name"`

### TSV (tab-separated)
```
marker	target
DAPI	Nucleus
CD45	Immune
```
Use: `marker_column=0` or `marker_column="marker"`